# Basic Preprocessing Pipeline for Arabic Dataset

This notebook loads the Arabic training data, applies basic preprocessing (character normalization, diacritic removal), and saves the cleaned dataset.

In [1]:
import pandas as pd
import sys
import os

# Add parent directory to path to import preprocessing module
sys.path.insert(0, os.path.abspath('..'))

from ArbPreBasic import ArabicBasicPreprocessor

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [2]:
# Load the Arabic training dataset
train_path = '../train/arb_merged.csv'
df = pd.read_csv(train_path)

print(f"✓ Dataset loaded from: {train_path}")
print(f"  Shape: {df.shape}")
print(f"  Columns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

✓ Dataset loaded from: ../train/arb_merged.csv
  Shape: (6056, 3)
  Columns: ['id', 'text', 'polarization']

First few rows:


,id,text,polarization
0,arb_a2a60c8b4af3389e842d8ec31afb0eea,احلام انتي ونعالي ومنو انتي حتى تقيمين الفناني...,1
1,arb_6723e56a672674a6c1d9b28b213c4a05,وره الكواليس تنيجج من وره بعير صطناعي على فكرة...,1
2,arb_b0365d606edeee38ae6c025b1ca33e96,.خخخخ الملكه احلام فيها شذوذ شنو هل بوس والدلع...,1
3,arb_858c0ee684049ba6f416a6cecb0b0761,الله يخزي احلام هي والبرنامج الخايس الي كله مصخره,1
4,arb_bdafc73afd0bc2cd2badae2a089446b9,كس ام احلام الي ماربتها وش ملكه هههه متستاهل م...,1


In [3]:
# Check dataset info
print("Dataset Information:")
print(f"  Total records: {len(df)}")
print(f"  Null values:\n{df.isnull().sum()}")
print(f"\nPolarization distribution:")
print(df['polarization'].value_counts())

Dataset Information:
  Total records: 6056
  Null values:
id              0
text            0
polarization    0
dtype: int64

Polarization distribution:
polarization
0    3542
1    2514
Name: count, dtype: int64


In [4]:
# Initialize the basic preprocessor
preprocessor = ArabicBasicPreprocessor()

print("✓ Basic preprocessor initialized")
print("\nPreprocessor features:")
print("  • Character normalization (Alef, Hamza, Ya variants)")
print("  • Diacritic removal")
print("  • Tatweel removal")

# Test on a sample
sample = df['text'].iloc[0]
print(f"\n{'='*80}")
print("Sample preprocessing:")
print(f"{'='*80}")
print(f"Original:\n{sample[:100]}...")
print(f"\nPreprocessed:\n{preprocessor.preprocess(sample)[:100]}...")

✓ Basic preprocessor initialized

Preprocessor features:
  • Character normalization (Alef, Hamza, Ya variants)
  • Diacritic removal
  • Tatweel removal

Sample preprocessing:
Original:
احلام انتي ونعالي ومنو انتي حتى تقيمين الفنانين الملكه احلام هههههههه البقره احلام بابا عوفي الفن لا...

Preprocessed:
احلام انتي ونعالي ومنو انتي حتي تقيمين الفنانين الملكه احلام هههههههه البقره احلام بابا عوفي الفن لا...


In [5]:
# Apply preprocessing to all texts
print(f"Preprocessing {len(df)} texts...")

df['text_clean'] = df['text'].apply(lambda x: preprocessor.preprocess(x) if pd.notna(x) else x)

print(f"✓ Preprocessing complete!")
print(f"\nDataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

Preprocessing 6056 texts...
✓ Preprocessing complete!

Dataset shape: (6056, 4)
Columns: ['id', 'text', 'polarization', 'text_clean']


In [6]:
# Compare original vs preprocessed
print("Comparison: Original vs Preprocessed\n")
print("="*100)

for i in range(3):
    print(f"\nExample {i+1}:")
    print(f"  Original:  {df['text'].iloc[i][:80]}...")
    print(f"  Clean:     {df['text_clean'].iloc[i][:80]}...")
    print(f"  Label:     {df['polarization'].iloc[i]}")
    print("-"*100)

# Statistics
orig_len = df['text'].str.len().mean()
clean_len = df['text_clean'].str.len().mean()
print(f"\nText Length Statistics:")
print(f"  Original average:     {orig_len:.2f} chars")
print(f"  Preprocessed average: {clean_len:.2f} chars")
print(f"  Reduction:            {(orig_len - clean_len) / orig_len * 100:.2f}%")

Comparison: Original vs Preprocessed


Example 1:
  Original:  احلام انتي ونعالي ومنو انتي حتى تقيمين الفنانين الملكه احلام هههههههه البقره احل...
  Clean:     احلام انتي ونعالي ومنو انتي حتي تقيمين الفنانين الملكه احلام هههههههه البقره احل...
  Label:     1
----------------------------------------------------------------------------------------------------

Example 2:
  Original:  وره الكواليس تنيجج من وره بعير صطناعي على فكرة احﻻم رجل مو مره لهن تخيل على البن...
  Clean:     وره الكواليس تنيجج من وره بعير صطناعي علي فكرة احﻻم رجل مو مره لهن تخيل علي البن...
  Label:     1
----------------------------------------------------------------------------------------------------

Example 3:
  Original:  .خخخخ الملكه احلام فيها شذوذ شنو هل بوس والدلع مع شذا والله عيب اطلعت بويه احلام...
  Clean:     .خخخخ الملكه احلام فيها شذوذ شنو هل بوس والدلع مع شذا والله عيب اطلعت بويه احلام...
  Label:     1
-------------------------------------------------------------------------------------------------

In [7]:

# Also create a clean version with only the preprocessed text
df_clean = df[['id', 'text_clean', 'polarization']].copy()
df_clean.rename(columns={'text_clean': 'text'}, inplace=True)

clean_output_path = '../train/arb_merged_clean_basic.csv'
df_clean.to_csv(clean_output_path, index=False)

print(f"\n✓ Clean version saved!")
print(f"  Location: {clean_output_path}")
print(f"  Shape: {df_clean.shape}")
print(f"\n{'='*80}")
print("SUMMARY")
print(f"{'='*80}")
print(f"Processed {len(df)} records")


✓ Clean version saved!
  Location: ../train/arb_merged_clean_basic.csv
  Shape: (6056, 3)

SUMMARY
Processed 6056 records
